In [78]:
"""Build index for ratchet_core docs"""
import os
from dotenv import load_dotenv
from langchain.agents import Tool, initialize_agent, AgentExecutor, AgentType
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from llama_index import GPTSimpleVectorIndex, download_loader


def get_env():
    """Load environment variables from .env file"""
    load_dotenv()
    os.getenv("OPENAI_API_KEY")


def build_and_save_index(crate_name: str):
    """Build and save index for ratchet_core docs"""
    simple_directory_loader = download_loader("SimpleDirectoryReader")

    documents = simple_directory_loader(
        input_dir=f"../embedding-docs/{crate_name}",
        recursive=True,
        file_extractor={
            ".html": "UnstructuredReader",
        },
    ).load_data()

    index = GPTSimpleVectorIndex.from_documents(documents)
    index.save_to_disk(f"../embedding-indices/{crate_name}_index.json")


def load_index(crate_name: str):
    """Load index for ratchet_core docs"""
    index = GPTSimpleVectorIndex.load_from_disk(
        f"../embedding-indices/{crate_name}_index.json"
    )
    return index


def build_chat_agent(crate_name: str):
    """Build chat agent for ratchet_core docs"""
    index = load_index(crate_name)
    memory = ConversationBufferMemory(memory_key=f"{crate_name}_memory")
    llm = ChatOpenAI(temperature=0.3)
    tools = [
        Tool(
            name=f"{crate_name} docs and source code",
            func=lambda q: str(index.query(q)),
            description=f"Useful for everything involving {crate_name} - the rust crate in context -  docs. The input to this tool should be a complete english sentence.\n",
            return_direct=True,
        )
    ]
    agent_chain = initialize_agent(
        tools, llm, agent="conversational-react-description", memory=memory
    )
    return agent_chain


In [89]:
from langchain import OpenAI


crate_name = "actix-web"
index = load_index(crate_name)
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(temperature=0.3)
tools = [
    Tool(
        name=f"{crate_name} crate documentation and source code",
        func=lambda q: str(index.query(q)),
        description=f"Useful for when you need to answer questions about {crate_name}. The input to this tool should be a complete english sentence.\n",
    )
]
agent_chain = initialize_agent(
    tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory
)

In [90]:
agent_chain.run("In what scenarios would I use this crate?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: The actix-web crate can be used in a variety of scenarios, such as building web applications, creating web services, and creating web APIs. It is designed to be a lightweight, high-performance web framework that is easy to use and extend.

> Finished chain.


'The actix-web crate can be used in a variety of scenarios, such as building web applications, creating web services, and creating web APIs. It is designed to be a lightweight, high-performance web framework that is easy to use and extend.'

In [91]:
agent_chain.run("What are all the ways that I can initialize a route?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: actix-web crate documentation and source code
Action Input: How can I initialize a route?

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 99 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 7 tokens



Observation: 
To initialize a route, you can use the Route::new() method. This method takes a path and a handler as arguments and returns a Route object. For example: let route = Route::new("/", handler);
Thought: Do I need to use a tool? No
AI: You can also use the App::route() method, which takes a path and a closure as arguments and returns a Route object. For example: let route = app.route("/", |req| { /* handler code here */ });

> Finished chain.


'You can also use the App::route() method, which takes a path and a closure as arguments and returns a Route object. For example: let route = app.route("/", |req| { /* handler code here */ });'

In [92]:
agent_chain.run("What other ways can I do that?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: You can also use the App::route_with_options() method, which takes a path, a closure, and an Options struct as arguments and returns a Route object. This allows you to specify additional options such as the HTTP method and the request guard. For example: let route = app.route_with_options("/", |req| { /* handler code here */ }, Options::new().method(Method::GET));

> Finished chain.


'You can also use the App::route_with_options() method, which takes a path, a closure, and an Options struct as arguments and returns a Route object. This allows you to specify additional options such as the HTTP method and the request guard. For example: let route = app.route_with_options("/", |req| { /* handler code here */ }, Options::new().method(Method::GET));'